# Testing Run Length Encoding


## Imports


In [1]:
from collections import deque
from importlib.util import spec_from_loader, module_from_spec
from importlib.machinery import SourceFileLoader
from scipy.io import wavfile
from glob import glob
import numpy as np

# Import Encode
spec = spec_from_loader("encode", SourceFileLoader("encode", "../../encode"))
encode = module_from_spec(spec)
spec.loader.exec_module(encode)

# Import Decode
spec = spec_from_loader("decode", SourceFileLoader("decode", "../../decode"))
decode = module_from_spec(spec)
spec.loader.exec_module(decode)

In [2]:
data_dir = "../../data/"
data_file_l = glob(data_dir + "*.wav")
current_file = data_file_l[0]

In [3]:
rate, data = wavfile.read(current_file)

In [4]:
data_bytes = data.tobytes()

In [5]:
compressed_file_path = "../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire"

In [6]:
encode.huffman_encoding(
    input_data=data_bytes,
    compressed_file_path="../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire",
)

In [7]:
with open(compressed_file_path, "rb") as fp:
    data_huffman_encoded_bytes = fp.read()
    fp.close()

In [8]:
type(data_huffman_encoded_bytes)

bytes

In [9]:
for byte in data_huffman_encoded_bytes[:12]:
    print(byte)

128
4
149
135
5
0
0
0
0
0
0
125


In [10]:
data_huffman_encoded_bytes_sub_string = data_huffman_encoded_bytes[:12]

In [11]:
rle_l_raw = []
for data in data_huffman_encoded_bytes_sub_string:
    rle_l_raw.append(data)
rle_l_raw = deque(rle_l_raw)

In [12]:
rle_l_encoded = []

In [13]:
rle_l_raw

deque([128, 4, 149, 135, 5, 0, 0, 0, 0, 0, 0, 125])

In [14]:
def rle_algorithm(rle_l_raw: list):
    """This algorithm will search for contiguous values within the
       array.

    Args:
        rle_l_raw (list): This is a list of integer values to be
                          encoded.

    Returns:
        index_array (list): This is the list of run length encoded
                            values.
    """
    initial_index = 0
    second_index = 1
    count = 0
    index_array = []

    while second_index < len(rle_l_raw):
        if rle_l_raw[initial_index] == rle_l_raw[second_index]:
            index_array.append(rle_l_raw[initial_index])
            # continue searching the breadth of the array; increasing count
            while (
                second_index <= len(rle_l_raw)
                and rle_l_raw[initial_index] == rle_l_raw[second_index]
            ):
                count += 1
                second_index += 1
            index_array.append(count)
        else:
            index_array.append(rle_l_raw[initial_index])
            index_array.append(1)
        count = 0
        initial_index = second_index
        second_index += 1
    if rle_l_raw[-1] != index_array[-2]:
        index_array.append(rle_l_raw[-1])
        index_array.append(1)
    return index_array

In [17]:
index_array = rle_algorithm(rle_l_raw)

In [16]:
index_array

[128, 1, 4, 1, 149, 1, 135, 1, 5, 1, 0, 5, 125, 1]